## **Single Stage**

### Real Data

**Mimic3**: https://www.kaggle.com/datasets/asjad99/mimiciii


Mimic3 is a large open-access anonymized single-center database which consists of comprehensive clinical data of 61,532 critical care admissions from 2001–2012 collected at a Boston teaching hospital. Dataset consists of 47 features (including demographics, vitals, and lab test results) on a cohort of sepsis patients who meet the sepsis-3 definition criteria.

In causal effect learning, we try to estimate the treatment effect of conducting a specific intervention (e.g use of ventilator) to the patient, either given a particular patient’s characteristics and physiological information, or evaluate all patients treatment effect as a whole.

The original Mimic3 data was loaded from mimic3_sepsis_data.csv. For illustration purpose, we selected several representative features for the following analysis. 




#### Data Pre-processing

In [1]:
# import related packages
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt;
from lightgbm import LGBMRegressor;
from sklearn.linear_model import LinearRegression
#from causaldm.data import mimic3_sepsis_data

OSError: dlopen(/Users/alinaxu/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: '/usr/local/opt/libomp/lib/libomp.dylib'
  Referenced from: '/Users/alinaxu/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so'
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)

In [ ]:
# Get data

mimic3_data = pd.read_csv("C:/Users/Public/CausalDM/causaldm/data/mimic3_sepsis_data.csv")
mimic3_data

,bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,...,input_total,input_4hourly,output_total,output_4hourly,cumulated_balance,SOFA,SIRS,vaso_input,iv_input,reward
0,1,3,7245486000,0,17639.826435,0,0,0,0,1,...,6527.000,50.0,13617.0,520.0,-7090.000,5,1,0.0,2.0,-0.884898
1,1,11,6898241400,1,30766.069028,6,1,0,0,0,...,0.000,0.0,0.0,0.0,0.000,12,0,0.0,0.0,0.383136
2,1,12,5805732000,1,12049.217303,0,0,0,0,0,...,0.000,0.0,0.0,0.0,0.000,4,2,0.0,0.0,0.976040
3,1,14,4264269300,0,30946.970000,2,0,0,0,1,...,1300.000,1300.0,340.0,160.0,960.000,5,2,0.0,4.0,0.125000
4,1,30,5707825200,0,19793.588912,6,0,0,0,0,...,9552.000,50.0,6830.0,540.0,2722.000,6,2,0.0,2.0,0.457625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20950,1,99961,4379511600,0,18316.820324,7,0,0,0,0,...,7665.000,0.0,11314.0,0.0,-3649.000,6,2,0.0,0.0,-0.025000
20951,1,99984,6807542400,1,28815.459329,3,1,0,0,0,...,9572.600,21.3,15517.0,605.0,-5944.400,10,3,4.0,1.0,-0.690146
20952,1,99988,6946088400,0,11978.343576,0,0,0,0,0,...,30918.650,57.0,19835.0,480.0,11083.650,5,3,0.0,2.0,1.773188
20953,1,99992,4289137200,0,15191.885116,0,0,0,0,0,...,14948.442,91.0,30256.0,950.0,-15307.558,13,3,3.0,2.0,0.250000


In [ ]:
selected = ['Glucose','paO2','PaO2_FiO2',  'iv_input', 'SOFA','reward']
n = 5000
mimic3_data_selected = mimic3_data[:n][selected]
mimic3_data_selected

,Glucose,paO2,PaO2_FiO2,iv_input,SOFA,reward
0,84.000000,84.000000,168.000000,2.0,5,-0.884898
1,122.000000,59.444444,198.148148,0.0,12,0.383136
2,125.000000,192.000000,690.647482,0.0,4,0.976040
3,110.727273,179.000000,447.499993,4.0,5,0.125000
4,187.000000,125.000000,347.222222,2.0,6,0.457625
...,...,...,...,...,...,...
4995,121.375000,136.787683,206.005547,3.0,4,-1.965110
4996,108.000000,62.333333,143.846153,0.0,11,-0.025000
4997,106.000000,258.500000,923.214286,0.0,7,0.402531
4998,144.000000,376.000000,752.000000,1.0,4,-0.172130


In [ ]:
userinfo_index = np.array([0,1,2,4]) # record all the indices of patients' information
SandA = mimic3_data_selected.iloc[:, np.array([0,1,2,3,4])]

data_CEL_selected = mimic3_data_selected
data_CEL_selected.iloc[np.where(mimic3_data_selected['iv_input']!=0)[0],:] = 1
# change the discrete action to binary
data_CEL_selected

,Glucose,paO2,PaO2_FiO2,iv_input,SOFA,reward
0,1.0,1.000000,1.000000,1.0,1,1.000000
1,122.0,59.444444,198.148148,0.0,12,0.383136
2,125.0,192.000000,690.647482,0.0,4,0.976040
3,1.0,1.000000,1.000000,1.0,1,1.000000
4,1.0,1.000000,1.000000,1.0,1,1.000000
...,...,...,...,...,...,...
4995,1.0,1.000000,1.000000,1.0,1,1.000000
4996,108.0,62.333333,143.846153,0.0,11,-0.025000
4997,106.0,258.500000,923.214286,0.0,7,0.402531
4998,1.0,1.000000,1.000000,1.0,1,1.000000


#### Final Mimic3 Data Selected for Causal Effect Learning (CEL)

After pre-processing, we selected 4 features as the state variable in CEL, which represents the baseline information of the patients:
*   **Glucose**:  glucose values of patients
*   **paO2**: The partial pressure of oxygen
*   **PaO2_FiO2**: The partial pressure of oxygen (PaO2)/fraction of oxygen delivered (FIO2) ratio.
*   **SOFA**: Sepsis-related Organ Failure Assessment score to describe organ dysfunction/failure.

The action variable is **iv-input**, which denotes the volumn of fluids that have been administered to the patient. Additionally, we set all non-zero iv-input values as $1$ to create a binary action space.

The last column denotes the reward we evaluated according to the status of patients from several aspects.

